## Module to scrap data from API based on Gspread provided

In [1]:
'''Getting the input values from Google Spread API'''

import gspread
import pandas as pd

def gspreadData():
    sheet_id='1jYrsjlx4xD8MSkFGM3mItJT59ytoc5_LZJgE6wMSWK0'
    sheet_name='CovidData'
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    url  
    df=pd.read_csv(url,error_bad_lines=False)
    df.dropna(axis=1, how='all')
    print(df)
    return df



In [10]:
'''Scrapping value from CoWInAPI'''

import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs
import datetime
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

##this above import is to minismise SSL Security warning in Macbook


'''
Scripts to get the Vaccination available slot for covid-19 centers with ease 

Open public API: https://apisetu.gov.in/public/marketplace/api/cowin
'''

base = datetime.datetime.today()
print(base)

numdays = 25
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
print(date_list)
date_str = [x.strftime("%d-%m-%Y") for x in date_list]
date_str
# age=58
print_flag = 'y'
print_flag = 'y'

headers={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

def vaccine_state_data():
    '''function to print the state and its correponsding district names'''

    try:
        ## Checking statecode avaialble
        response_state = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/states",headers=headers,verify=False)
        json_data_state = json.loads(response_state.text)
#         print()
        json_data_state['states']

        data_dist=[]
        for state_id in range(1,40):
            ##Checking till 40 state code as i checked there are 40 statecode availble!
#             try:
#                 print("For State_code:{} State_name: {} ".format(state_id,next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id)))
#             except:
#                 print("For State_code:{} State_name: {}".format(state_id,'Not Available') )
#                 pass
            response_dist = rq.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id),headers=headers,verify=False)
            json_data = json.loads(response_dist.text)
            for _ in json_data["districts"]:
                
                # print(_["district_id"],'\t', _["district_name"])
                try:
                    data_dist.append([_["district_id"], _["district_name"],next(i['state_name'] for i in json_data_state['states'] if i['state_id']==state_id),state_id])
                except:
                    data_dist.append([_["district_id"], _["district_name"],None,state_id])
            # print("\n")

        df = pd.DataFrame(data_dist, columns = ['district_id', 'district_name','State_name','State_id'],index=None)
        df=df.sort_values(by=['State_id'], ascending=True)
        df_vacc_state = df.reindex(columns=['State_id','State_name','district_name','district_id'])
#         print(len(df_vacc_state['district_id']))
        df_vacc_state['State_id'].nunique()
        return df_vacc_state


    except Exception as e:
        print("Exception ",e)



def vaccineSlotsByDist(age,district_id):
    try:
        available_df=[]
        age=age
        
        df_vacc_state=vaccine_state_data()
        try:
            print(df_vacc_state)
            DIST_ID=district_id
            ##list(df_vacc_state[(df_vacc_state['State_name']== state) & (df_vacc_state['district_name']==district_name)]['district_id'])[0]

        except:
            print("No Combintaiont found")
            return None
            pass

        for inp_date in date_str:
            URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, inp_date)
            response = rq.get(URL,headers=headers,verify=False)
            if response.ok:
                resp_json = response.json()
                # print(json.dumps(resp_json, indent = 1))
                if resp_json["centers"]:
#                     print("Available on: {}".format(inp_date))
                    if(print_flag=='y' or print_flag=='Y'):
                        for center in resp_json["centers"]:
                            for session in center["sessions"]:
                                if session["min_age_limit"] <= age :
#                                     print(session["min_age_limit"])
#                                     print("\t", center["name"])
#                                     print("\t", center["block_name"])
#                                     print("\t Price: ", center["fee_type"])
#                                     print("\t Available Capacity: ", session["available_capacity"])
                                    if(session["vaccine"] != ''):
#                                         print("\t Vaccine: ", session["vaccine"])
                                        pass
                                        
                                    else:
                                        
                                        session["vaccine"]='No Information'
#                                     print("\n\n")
                                    available_df.append([center["name"],center["block_name"],center["fee_type"],session["vaccine"],\
                                                        center['district_name'],center['state_name'],\
                                                        
                                                        center['pincode'],inp_date, session["available_capacity"], session["min_age_limit"]
                                                        
                                                        ])
                                                        
                           
                                 

                        
                else:
                    pass
#                     print("No available slots on {}".format(inp_date))




        # Create the pandas DataFrame for re grouping whole data into one particlur data frames
        info_vaccination = pd.DataFrame(available_df, columns = ['Vaccination Center', 'Block_name','Fee_type','Vaccine Type',\
            'District','State Name','Pincode','Available Date','available capacity','Age Group'],index=None)
        
        indexNames = info_vaccination[ (info_vaccination['available capacity'] == 0) ].index
    
        info_vaccination.drop(indexNames , inplace=True)
       
        html=info_vaccination#.set_index(['Available Date', 'State Name','District','pincode'])
        print(html)
        return html

    except Exception as e:
        print("Exception ",e)




2021-05-13 09:48:44.691330
[datetime.datetime(2021, 5, 13, 9, 48, 44, 691330), datetime.datetime(2021, 5, 14, 9, 48, 44, 691330), datetime.datetime(2021, 5, 15, 9, 48, 44, 691330), datetime.datetime(2021, 5, 16, 9, 48, 44, 691330), datetime.datetime(2021, 5, 17, 9, 48, 44, 691330), datetime.datetime(2021, 5, 18, 9, 48, 44, 691330), datetime.datetime(2021, 5, 19, 9, 48, 44, 691330), datetime.datetime(2021, 5, 20, 9, 48, 44, 691330), datetime.datetime(2021, 5, 21, 9, 48, 44, 691330), datetime.datetime(2021, 5, 22, 9, 48, 44, 691330), datetime.datetime(2021, 5, 23, 9, 48, 44, 691330), datetime.datetime(2021, 5, 24, 9, 48, 44, 691330), datetime.datetime(2021, 5, 25, 9, 48, 44, 691330), datetime.datetime(2021, 5, 26, 9, 48, 44, 691330), datetime.datetime(2021, 5, 27, 9, 48, 44, 691330), datetime.datetime(2021, 5, 28, 9, 48, 44, 691330), datetime.datetime(2021, 5, 29, 9, 48, 44, 691330), datetime.datetime(2021, 5, 30, 9, 48, 44, 691330), datetime.datetime(2021, 5, 31, 9, 48, 44, 691330), dat

In [3]:
'''Twitter Updatation'''
import datetime;
import tweepy
import webbrowser
import time
from tweepy import OAuthHandler

import tweepy
from tweepy import StreamListener
from tweepy import Stream
import webbrowser

'''Hyderabad
Center 1 - Covishild( 45+ / 190 slots available ) ,  Covaxin( 18-44) / 190 slots available )

#COVID19IndiaHelp #VaxTrack

Last Tracked: Date and time'''
def tweet_data(msg):
##tweet

    CONSUMER_KEY='JTZluB3daESewe2ToGofBEdFG'
    CONSUMER_SECRET='hg3BGax9d5TiHw4UmB117WQYKE6Gnhe1dOL0zYyb4DfFlkH5HJ'
    Access_Token='1389917390520033284-of3M07i7EFEtwaM3GtoYkd62vSvaaA'
    Access_Token_Secret='z3ECIPDP6Lh7vbe0k1v0nYxer6GsvCnumsQ3Xtlsr7MqH'


    # callback_uri='oob'
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(Access_Token,Access_Token_Secret)
    api=tweepy.API(auth)
    
    print(dist)
    try:
        api.update_status("{}".format(msg))
        print("TWeet Successful******")
    except Exception as e:
        print("Exception in Tweeter API status Update: ",e)
    #val.destroy()
    

    



In [19]:
 gspread['DistId'].tolist()

[457, 446, 454, 447, 453, 581, 78, 140]

In [27]:
from datetime import datetime
import time
now = datetime.now()
tweet_count=0
if __name__=='__main__':
    print("hello")
    gspread_data=gspreadData()

    for i in gspread_data['DistId'].tolist():
        print(i)
        print("for District Name: ",gspread_data[gspread_data['DistId']==i]['DistName'])
        val_18=vaccineSlotsByDist(40,i)
        time.sleep(2)
        val_45=vaccineSlotsByDist(55,i)
        print(val_45,val_18)
        time.sleep(2)
        try:
            value=pd.concat([val_18, val_45], axis=0)
        except:
            print("Exception in Value concatination")
        print("Value:",value)
#         if value.empty:
#             print('DataFrame is empty!')
#             continue
        
        for inp_date in date_str:
            dist='_'.join(gspread_data[gspread_data['DistId']==i]['DistName'].tolist()[0].split(' '))
            
            msg="#{0} \n Date: {1} \n\n\t Slots Available: \n\t".format(dist,inp_date)
            print("String dates inserting",dist)
            
            covs_45=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVISHIELD') & (value['Age Group']==45)])
           
            covx_45=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVAXIN') & (value['Age Group']==45)])
        
            covs_18=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVISHIELD') & (value['Age Group']==18)])
           
            covx_18=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVAXIN') & (value['Age Group']==18)])
           
                
#             print([covs_45,covx_45,covs_18,covx_18])
            if all(v == 0 for v in [covs_45,covx_45,covs_18,covx_18]):
                print("No data for: ",inp_date)
#                 continue

            else:

                if covs_45 !=0:
                    cov_45_flag='\t Covishield - 45+/{} Slots \n\t'.format(covs_45)
                    msg=msg+cov_45_flag
                if covx_45 !=0:
                    covx_45_flag='\t Covaxin - 45+/{} Slots \n\t'.format(covx_45)
                    msg=msg+covx_45_flag

                if covs_18 !=0:
                    covs_18_flag='\t Covishield (18-44)/{} Slots \n\t'.format(covs_18)
                    msg=msg + covs_18_flag

                if covx_18 !=0:
                    covx_18_flag='\t Covaxin (18-44)/{} Slots \n\t'.format(covx_18)
                    msg=msg+ covx_18_flag

                msg=msg+'#COVID19IndiaHelp #VaxTrack \n\n Last Tracked: {}'.format(now.strftime('%d-%m-%Y, %H:%M%p'))
                print(msg)
                tweet_data(msg)
                tweet_count+=1
                

print(tweet_count)       
        
        
        
        

hello
   DistId    DistName  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0     457     Cuttack         NaN         NaN         NaN         NaN   
1     446      Khurda         NaN         NaN         NaN         NaN   
2     454     Bhadrak         NaN         NaN         NaN         NaN   
3     447    Balesore         NaN         NaN         NaN         NaN   
4     453  Sundargarh         NaN         NaN         NaN         NaN   
5     581   Hyderabad         NaN         NaN         NaN         NaN   
6      78       Arwal         NaN         NaN         NaN         NaN   
7     140   New Delhi         NaN         NaN         NaN         NaN   

   Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  Unnamed: 16  \
0         NaN         NaN         NaN         NaN  ...          NaN   
1         NaN         NaN         NaN         NaN  ...          NaN   
2         NaN         NaN         NaN         NaN  ...          NaN   
3         NaN         NaN         NaN         NaN  .

In [18]:
for i in [140]:
    print(i)
    print("for District Name: ",gspread[gspread['DistId']==i]['DistName'])
    val_18=vaccineSlotsByDist(40,i)
    val_45=vaccineSlotsByDist(55,i)
    print("dsds:", val_45,val_18)
    time.sleep(5)
    value=pd.concat([val_18, val_45], axis=0)
    print("Value:",value)
    if value.empty:
        print('DataFrame is empty!')
        continue

    for inp_date in date_str:
        dist='_'.join(value['District'].unique().tolist()[0].split(' '))

        msg="#{0} \n Date: {1} \n\n\t Slots Available: \n\t".format(dist,inp_date)
        covs_45=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVISHIELD') & (value['Age Group']==45)])
        covx_45=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVAXIN') & (value['Age Group']==45)])

        covs_18=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVISHIELD') & (value['Age Group']==18)])
        covx_18=len(value[(value['Available Date']==inp_date) & (value['Vaccine Type']=='COVAXIN') & (value['Age Group']==18)])
#             print([covs_45,covx_45,covs_18,covx_18])
        if all(v == 0 for v in [covs_45,covx_45,covs_18,covx_18]):
            print("No data for: ",inp_date)
            continue

        else:

            if covs_45 !=0:
                cov_45_flag='\t Covishield -45+/{} Slots \n\t'.format(covs_45)
                msg=msg+cov_45_flag
            if covx_45 !=0:
                covx_45_flag='\t Covaxin -45+/{} Slots \n\t'.format(covx_45)
                msg=msg+covx_45_flag

            if covs_18 !=0:
                covs_18_flag='\t Covishield (18-44)/{} Slots \n\t'.format(covs_18)
                msg=msg + covs_18_flag

            if covx_18 !=0:
                covx_18_flag='\t Covaxin (18-44)/{} Slots \n\t'.format(covx_18)
                msg=msg+ covx_18_flag

            msg=msg+'#COVID19IndiaHelp #VaxTrack \n\n Last Tracked: {}'.format(now.strftime('%d-%m-%Y, %H:%M%p'))
            print(msg)
            tweet_data(msg)

140
for District Name:  7    New Delhi
Name: DistName, dtype: object
     State_id                   State_name                    district_name  \
0           1  Andaman and Nicobar Islands                          Nicobar   
1           1  Andaman and Nicobar Islands         North and Middle Andaman   
2           1  Andaman and Nicobar Islands                    South Andaman   
15          2               Andhra Pradesh  YSR District, Kadapa (Cuddapah)   
13          2               Andhra Pradesh                     Vizianagaram   
..        ...                          ...                              ...   
759        39                         None                            Mandi   
756        39                         None                           Chamba   
757        39                         None                           Kangra   
758        39                         None                          Kinnaur   
760        39                         None                    